In [ ]:
import numpy as np 
import pandas as pd 
import warnings; warnings.filterwarnings(action='ignore') # 경고 메시지 무시
import matplotlib.pyplot as plt 

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split # 훈련 데이터, 테스트 데이터 분리
from sklearn.preprocessing import StandardScaler # 정규화
from sklearn.ensemble import RandomForestClassifier as RFC # 랜덤포레스트 분류 알고리즘
from sklearn.tree import DecisionTreeClassifier as DTC # 의사결정나무 분류 알고리즘



In [ ]:
bike_train=pd.read_csv('train.csv',encoding='CP949')
bike_test=pd.read_csv('test.csv',encoding='CP949')

#train과 test 자료의 구조 살펴보기
bike_train.shape
(1459, 11)
bike_test.shape
(715, 10)

In [ ]:
bike_train.head(10)
bike_test.head(10)

In [ ]:
#train 데이터의 결측치를 train데이터의 열의 평균으로 대체하였다. 
bike_train.fillna(bike_train.mean(),inplace=True)
bike_train.isnull().sum()
#test 데이터의 결측치를 test 데이터의 열의 평균으로 대체하였다. 
bike_test.fillna(bike_test.mean(),inplace=True)
bike_test.isnull().sum()

In [ ]:
def remove_outlier(df):
    q1 = df.quantile(0.25) # 제 1사분위수
    q3 = df.quantile(0.75) # 제 3사분위수
    iqr = q3 - q1 # IQR 계산
    minimum = q1 - (iqr * 1.5) # IQR 최솟값
    maximum = q3 + (iqr * 1.5) # IQR 최댓값
    # IQR 범위 내에 있는 데이터만 산출(IQR 범위 밖의 데이터는 이상치)
    df_removed = df[(minimum < df) & (df < maximum)]
    return df_removed

In [ ]:
#이상치를 제거한 df인 bike_train_r을 만들고 결측치를 제거하였다.
bike_train_r=remove_outlier(bike_train)
bike_train_r.isnull().sum()
#이 과정을 통해 이상치가 제거된 df인 btr을 만들었다.
btr=bike_train_r.dropna().reset_index(drop=True)
btr

In [ ]:
dfall=pd.merge(bike_train,btr,how='inner')
#hour_bef_precipitation은 비가오면 1 안오면 0의 자료였기 때문에 평균을 구해보았다.
bike_train.hour_bef_precipitation.mean()
->0.031571722717913524
#hour_bef_precipitation 의 결측치 수는 2개였기 때문에 결측치에 0을 넣기로 결정했다.
dfall.iloc[832,3]=0
dfall.iloc[833,3]=0